In [1]:
# Cell 1: Setup and Imports
import json
import time
from pathlib import Path
from typing import List, Dict
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv('../.env.local')

# Configure OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env.local")

client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI configured successfully")


✅ OpenAI configured successfully


In [2]:
# Cell 2: Load Questions Data
questions_file = Path('../all-questions-content.json')

if not questions_file.exists():
    print("❌ all-questions-content.json not found. Run scraping first!")
else:
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions = json.load(f)
    
    print(f"✅ Loaded {len(questions)} questions")
    print(f"\nSample question:")
    sample = questions[0]
    print(f"Name: {sample['name']}")
    print(f"Difficulty: {sample['difficulty']}")
    print(f"Tags: {sample['tags']}")
    print(f"Text preview: {sample['questionText'][:200]}...")


✅ Loaded 1210 questions

Sample question:
Name: Place Or Take
Difficulty: Hard
Tags: ['Probability']
Text preview: You are playing a one-player game with two opaque boxes. At each turn, you can choose to either "place" or "take". "Place" places $\$$1 from a third party into one box randomly. "Take" empties out one...


In [ ]:
# Cell 3: Define Fixed Categories

# Fixed categories - curated list
FIXED_CATEGORIES = [
    "Linear Algebra",
    "Uniform Random Variables",
    "Normal Random Variables",
    "Exponential Random Variables",
    "Hypergeometric Random Variables",
    "Binomial Random Variables",
    "Poisson Random Variables",
    "Continuous Random Variables",
    "Discrete Random Variables",
    "Coins",
    "Dice",
    "Cards",
    "Grids",
    "Martingales",
    "Markov Chains",
    "Stochastic Processes",
    "Random Walks",
    "Game Theory",
    "Calculus",
    "Geometry",
    "Algebraic Manipulation",
    "Combinatorics",
]

print(f"✅ Using {len(FIXED_CATEGORIES)} fixed categories:")
for i, cat in enumerate(FIXED_CATEGORIES, 1):
    print(f"  {i:2d}. {cat}")

# Categorization prompt
CATEGORIZATION_PROMPT = """You are categorizing quantitative finance interview questions.

## Available Categories:
{categories}

## Instructions:
1. Analyze the question name, tags, and text carefully
2. Assign AS MANY categories as applicable from the list above
3. **IMPORTANT**: If the question doesn't fit ANY category, return an empty array []
4. Only use categories from the provided list - do not create new ones
5. A question can have 0-10+ categories

## Examples:
- A question about flipping coins and calculating expected value → ["Coins", "Discrete Random Variables"]
- A question about matrices and eigenvalues → ["Linear Algebra"]
- A question about derivatives and integrals → ["Calculus"]
- A question about shortest paths on a grid → ["Grids", "Combinatorics"]
- A brainteaser that doesn't fit any category → []

## Output Format:
Return ONLY a JSON array of category strings. No explanation.
"""

print("\n✅ Categorization prompt defined")


✅ Using 22 fixed categories:
   1. Linear Algebra
   2. Uniform Random Variables
   3. Normal Random Variables
   4. Exponential Random Variables
   5. Hypergeometric Random Variables
   6. Binomial Random Variables
   7. Poisson Random Variables
   8. Continuous Random Variables
   9. Discrete Random Variables
  10. Coins
  11. Dice
  12. Cards
  13. Grids
  14. Martingales
  15. Markov Chains
  16. Stochastic Processes
  17. Random Walks
  18. Game Theory
  19. Calculus
  20. Geometry
  21. Algebraic Manipulation
  22. Combinatorics

✅ Categorization prompt defined


In [ ]:
# Cell 3.5: Fixed Categorization Function (Run this cell!)

def categorize_question(question: Dict, rate_limit_delay: float = 0.1) -> List[str]:
    """
    Categorize a single question using GPT-4o with strategy-focused analysis.
    """
    categories_str = "\n".join([f"- {cat}" for cat in FIXED_CATEGORIES])
    
    # Enhanced system prompt focusing on problem-solving strategies
    system_prompt = f"""You are an expert quantitative analyst categorizing finance interview questions. 

## CRITICAL: Focus on Solution Strategy, Not Just Keywords

**Your Task**: Analyze HOW you would actually solve this problem, then assign categories based on the mathematical techniques and concepts REQUIRED for the solution.

## Available Categories:
{categories_str}

## Analysis Process:
1. **Read the problem carefully** - What is it actually asking?
2. **Think through the solution** - What mathematical approach would you use?
3. **Identify required techniques** - What specific methods/distributions/concepts are needed?
4. **Assign categories** - Only assign categories for techniques actually used in the solution

## Category Guidelines:

**Random Variables** (assign ONLY if the solution requires these specific distributions):
- **Binomial**: Independent trials with fixed success probability
- **Uniform**: Equal probability over a range/set
- **Normal**: Gaussian distribution, CLT applications
- **Exponential**: Memoryless property, waiting times
- **Poisson**: Rare events, arrival processes
- **Hypergeometric**: Sampling without replacement
- **Discrete/Continuous**: Use only if no specific distribution applies

**Objects** (assign if problem involves these physical items):
- **Coins**: Coin flips/tosses
- **Dice**: Dice rolls
- **Cards**: Playing cards, deck problems
- **Grids**: Lattice paths, grid navigation

**Advanced Concepts** (assign only if solution requires these):
- **Martingales**: Expected value property used
- **Markov Chains**: State transitions, memoryless property
- **Random Walks**: Sum of random steps
- **Stochastic Processes**: Time-dependent random processes

**Mathematical Tools**:
- **Combinatorics**: Counting, permutations, combinations
- **Calculus**: Derivatives, integrals, optimization
- **Linear Algebra**: Matrices, eigenvalues, linear systems
- **Game Theory**: Strategic decision-making, Nash equilibrium
- **Geometry**: Shapes, areas, volumes, distances
- **Algebraic Manipulation**: Equation solving, algebraic simplification

## Output: JSON array only, no explanation."""
    
    user_prompt = f"""Question: {question['name']}
Difficulty: {question['difficulty']}
Original Tags: {', '.join(question['tags'])}

Problem Text:
{question['questionText'][:1000]}

Instructions:
1. Think: "How would I solve this problem step-by-step?"
2. Identify: "What mathematical techniques does my solution require?"
3. Categorize: Assign categories based on solution methods, not just keywords
4. Return: JSON array of applicable categories (or [] if none fit)

Categories:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1,
            max_tokens=500,
        )
        
        response_text = response.choices[0].message.content.strip()
        
        # Clean up response text
        if response_text.startswith('```'):
            lines = response_text.split('\n')
            response_text = '\n'.join(lines[1:-1]) if len(lines) > 2 else response_text
            if response_text.startswith('json'):
                response_text = response_text[4:].strip()
        
        # Parse JSON
        categories = json.loads(response_text)
        
        # Validate categories are from fixed list
        valid_categories = [cat for cat in categories if cat in FIXED_CATEGORIES]
        
        # Rate limiting
        time.sleep(rate_limit_delay)
        
        return valid_categories
    
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error for '{question['name']}': {e}")
        return []
    except Exception as e:
        print(f"❌ Error categorizing '{question['name']}': {e}")
        return []

print("✅ Fixed GPT-4o categorization function loaded!")


✅ Fixed GPT-4o categorization function loaded!


In [4]:
# Cell 3.6: Rate-Limited Categorization Function (Use This One!)

def categorize_question(question: Dict, rate_limit_delay: float = 1.0) -> List[str]:
    """
    Categorize a single question using GPT-4o with proper rate limiting.
    """
    categories_str = "\n".join([f"- {cat}" for cat in FIXED_CATEGORIES])
    
    # Enhanced system prompt focusing on problem-solving strategies
    system_prompt = f"""You are an expert quantitative analyst categorizing finance interview questions. 

## CRITICAL: Focus on Solution Strategy, Not Just Keywords

**Your Task**: Analyze HOW you would actually solve this problem, then assign categories based on the mathematical techniques and concepts REQUIRED for the solution.

## Available Categories:
{categories_str}

## Analysis Process:
1. **Read the problem carefully** - What is it actually asking?
2. **Think through the solution** - What mathematical approach would you use?
3. **Identify required techniques** - What specific methods/distributions/concepts are needed?
4. **Assign categories** - Only assign categories for techniques actually used in the solution

## Category Guidelines:

**Random Variables** (assign ONLY if the solution requires these specific distributions):
- **Binomial**: Independent trials with fixed success probability
- **Uniform**: Equal probability over a range/set
- **Normal**: Gaussian distribution, CLT applications
- **Exponential**: Memoryless property, waiting times
- **Poisson**: Rare events, arrival processes
- **Hypergeometric**: Sampling without replacement
- **Discrete/Continuous**: Use only if no specific distribution applies

**Objects** (assign if problem involves these physical items):
- **Coins**: Coin flips/tosses
- **Dice**: Dice rolls
- **Cards**: Playing cards, deck problems
- **Grids**: Lattice paths, grid navigation

**Advanced Concepts** (assign only if solution requires these):
- **Martingales**: Expected value property used
- **Markov Chains**: State transitions, memoryless property
- **Random Walks**: Sum of random steps
- **Stochastic Processes**: Time-dependent random processes

**Mathematical Tools**:
- **Combinatorics**: Counting, permutations, combinations
- **Calculus**: Derivatives, integrals, optimization
- **Linear Algebra**: Matrices, eigenvalues, linear systems
- **Game Theory**: Strategic decision-making, Nash equilibrium
- **Geometry**: Shapes, areas, volumes, distances
- **Algebra**: Equation solving, algebraic simplification

## Output: JSON array only, no explanation."""
    
    user_prompt = f"""Question: {question['name']}
Difficulty: {question['difficulty']}
Original Tags: {', '.join(question['tags'])}

Problem Text:
{question['questionText'][:1000]}

Instructions:
1. Think: "How would I solve this problem step-by-step?"
2. Identify: "What mathematical techniques does my solution require?"
3. Categorize: Assign categories based on solution methods, not just keywords
4. Return: JSON array of applicable categories (or [] if none fit)

Categories:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1,
            max_tokens=500,
        )
        
        response_text = response.choices[0].message.content.strip()
        
        # Clean up response text
        if response_text.startswith('```'):
            lines = response_text.split('\n')
            response_text = '\n'.join(lines[1:-1]) if len(lines) > 2 else response_text
            if response_text.startswith('json'):
                response_text = response_text[4:].strip()
        
        # Parse JSON
        categories = json.loads(response_text)
        
        # Validate categories are from fixed list
        valid_categories = [cat for cat in categories if cat in FIXED_CATEGORIES]
        
        # Proper rate limiting - wait AFTER successful request
        time.sleep(rate_limit_delay)
        
        return valid_categories
    
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error for '{question['name']}': {e}")
        time.sleep(rate_limit_delay * 2)  # Longer delay on error
        return []
    except Exception as e:
        print(f"❌ Error categorizing '{question['name']}': {e}")
        if "rate limit" in str(e).lower():
            print("   ⏳ Rate limit hit - waiting longer...")
            time.sleep(rate_limit_delay * 5)  # Much longer delay for rate limits
        else:
            time.sleep(rate_limit_delay * 2)
        return []

print("✅ Rate-limited GPT-4o categorization function loaded!")


✅ Rate-limited GPT-4o categorization function loaded!


In [ ]:
# Cell 4: Categorization Function

def categorize_question(question: Dict, rate_limit_delay: float = 0.1) -> List[str]:
    """
    Categorize a single question using fixed categories.
    
    Args:
        question: Question dict with name, tags, questionText
        rate_limit_delay: Seconds to wait between API calls
    
    Returns:
        List of category strings from fixed list (can be empty)
    """
    categories_str = "\n".join([f"- {cat}" for cat in FIXED_CATEGORIES])
    system_prompt = CATEGORIZATION_PROMPT.format(categories=categories_str)
    
    user_prompt = f"""Question Name: {question['name']}
Existing Tags: {', '.join(question['tags'])}
Difficulty: {question['difficulty']}

Question Text:
{question['questionText'][:800]}

Assign this question to appropriate categories (or return [] if none fit)."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
            max_tokens=1200,
        )
        
        response_text = response.choices[0].message.content.strip()
        
        # Remove markdown code blocks if present
        if response_text.startswith('```'):
            response_text = response_text.split('```')[1]
            if response_text.startswith('json'):
                response_text = response_text[4:]
        
        categories = json.loads(response_text.strip())
        
        # Validate categories are from fixed list
        valid_categories = [cat for cat in categories if cat in FIXED_CATEGORIES]
        
        # Rate limiting
        time.sleep(rate_limit_delay)
        
        return valid_categories
    
    except Exception as e:
        print(f"❌ Error categorizing '{question['name']}': {e}")
        return []

print("✅ Categorization function defined")


✅ Enhanced categorization function defined (GPT-4o with strategy focus)


In [5]:
# Cell 5: Test Categorization on Sample Questions

# Test on 5 sample questions
if 'questions' in locals():
    print("🧪 Testing categorization on 5 sample questions...\n")
    
    test_questions = questions[:5]
    
    for i, q in enumerate(test_questions, 1):
        print(f"\n{'='*80}")
        print(f"Question {i}: {q['name']}")
        print(f"Difficulty: {q['difficulty']} | Original Tags: {', '.join(q['tags'])}")
        print(f"Text: {q['questionText'][:150]}...")
        print(f"\n🤖 AI Categories:")
        
        categories = categorize_question(q, rate_limit_delay=0.5)
        if categories:
            for cat in categories:
                print(f"   ✓ {cat}")
        else:
            print(f"   (no matching categories)")
    
    print(f"\n{'='*80}")
    print("✅ Test complete!")
else:
    print("⚠️  Run Cell 2 first to load questions")


🧪 Testing categorization on 5 sample questions...


Question 1: Place Or Take
Difficulty: Hard | Original Tags: Probability
Text: You are playing a one-player game with two opaque boxes. At each turn, you can choose to either "place" or "take". "Place" places $\$$1 from a third p...

🤖 AI Categories:
   ✓ Game Theory
   ✓ Combinatorics

Question 2: Collecting Toys II
Difficulty: Hard | Original Tags: Probability
Text: Every box of cereal contains one toy from a group of 555 distinct toys, each of which is mutually independent from the others and is equally likely to...

🤖 AI Categories:
   ✓ Combinatorics
   ✓ Discrete Random Variables

Question 3: Chess Tournament I
Difficulty: Medium | Original Tags: Probability
Text: A chess tournament has 128 players, each with a distinct rating. Assume that the player with the higher rating always wins against a lower rated oppon...

🤖 AI Categories:
   ✓ Combinatorics

Question 4: Poker Hands I
Difficulty: Easy | Original Tags: Probability
Text: 

In [6]:
# Cell 6: Batch Categorization Function for OpenAI

def categorize_all_questions(
    questions: List[Dict],
    rate_limit_delay: float = 0.1,
    save_every: int = 50
) -> List[Dict]:
    """
    Categorize all questions with progress tracking and periodic saving.
    
    Args:
        questions: List of all questions
        rate_limit_delay: Seconds between API calls (OpenAI has higher limits)
        save_every: Save progress every N questions
    
    Returns:
        List of questions with added 'aiCategories' field
    """
    from datetime import datetime
    
    categorized = []
    total = len(questions)
    start_time = datetime.now()
    
    print(f"🚀 Starting GPT-4o categorization of {total} questions...")
    print(f"   Rate limit: {rate_limit_delay}s per request")
    print(f"   Estimated time: {total * rate_limit_delay / 60:.1f} minutes")
    print(f"   Model: GPT-4o with strategy-focused analysis\n")
    
    for i, question in enumerate(questions, 1):
        try:
            categories = categorize_question(question, rate_limit_delay=rate_limit_delay)
            
            categorized_q = question.copy()
            categorized_q['aiCategories'] = categories
            categorized.append(categorized_q)
            
            # Progress update
            if i % 25 == 0 or i == total:
                elapsed = (datetime.now() - start_time).total_seconds()
                rate = i / elapsed if elapsed > 0 else 0
                eta = (total - i) / rate if rate > 0 else 0
                
                print(f"Progress: {i:4d}/{total} ({i/total*100:5.1f}%) | "
                      f"Rate: {rate*60:5.1f} q/min | "
                      f"ETA: {eta/60:5.1f} min")
            
            # Periodic save
            if i % save_every == 0:
                with open('openai_categorized_progress.json', 'w', encoding='utf-8') as f:
                    json.dump(categorized, f, indent=2, ensure_ascii=False)
                print(f"   💾 Progress saved ({i} questions)")
        
        except Exception as e:
            print(f"❌ Error on question {i} ('{question['name']}'): {e}")
            # Add with empty categories
            categorized_q = question.copy()
            categorized_q['aiCategories'] = []
            categorized.append(categorized_q)
    
    elapsed = (datetime.now() - start_time).total_seconds()
    print(f"\n✅ Categorization complete!")
    print(f"   Total time: {elapsed/60:.1f} minutes")
    print(f"   Average: {elapsed/total:.2f}s per question")
    print(f"   Model: GPT-4o")
    
    return categorized

print("✅ Batch categorization function defined")


✅ Batch categorization function defined


In [7]:
# Cell 7: Rate-Limited Batch Categorization (Save Every 5)

def categorize_all_questions_safe(
    questions: List[Dict],
    rate_limit_delay: float = 1.5,  # Conservative 1.5 seconds between requests
    save_every: int = 5,            # Save every 5 questions
    start_from: int = 0             # Resume from specific question index
) -> List[Dict]:
    """
    Categorize all questions with aggressive rate limiting and frequent saves.
    
    Args:
        questions: List of all questions
        rate_limit_delay: Seconds between API calls (conservative for rate limits)
        save_every: Save progress every N questions
        start_from: Question index to start from (for resuming)
    
    Returns:
        List of questions with added 'aiCategories' field
    """
    from datetime import datetime
    
    # Load existing progress if available
    progress_file = 'openai_categorized_progress.json'
    if Path(progress_file).exists() and start_from == 0:
        try:
            with open(progress_file, 'r', encoding='utf-8') as f:
                existing_progress = json.load(f)
            print(f"📂 Found existing progress: {len(existing_progress)} questions")
            
            # Ask if user wants to resume
            start_from = len(existing_progress)
            categorized = existing_progress.copy()
        except:
            categorized = []
    else:
        categorized = []
    
    total = len(questions)
    start_time = datetime.now()
    
    print(f"🚀 Starting GPT-4o categorization...")
    print(f"   Questions: {start_from}/{total} (starting from {start_from})")
    print(f"   Rate limit: {rate_limit_delay}s per request (conservative)")
    print(f"   Save frequency: Every {save_every} questions")
    print(f"   Estimated time: {(total - start_from) * rate_limit_delay / 60:.1f} minutes")
    print(f"   Model: GPT-4o with strategy-focused analysis\n")
    
    for i in range(start_from, total):
        question = questions[i]
        question_num = i + 1
        
        try:
            print(f"Processing {question_num}/{total}: {question['name'][:50]}...")
            
            categories = categorize_question(question, rate_limit_delay=rate_limit_delay)
            
            categorized_q = question.copy()
            categorized_q['aiCategories'] = categories
            
            # Update or append
            if i < len(categorized):
                categorized[i] = categorized_q
            else:
                categorized.append(categorized_q)
            
            print(f"   ✅ Categories: {categories}")
            
            # Progress update every 10 questions
            if question_num % 10 == 0:
                elapsed = (datetime.now() - start_time).total_seconds()
                processed = question_num - start_from
                rate = processed / elapsed if elapsed > 0 else 0
                eta = (total - question_num) / rate if rate > 0 else 0
                
                print(f"\n📊 Progress: {question_num}/{total} ({question_num/total*100:5.1f}%)")
                print(f"   Rate: {rate*60:5.1f} q/min | ETA: {eta/60:5.1f} min\n")
            
            # Save frequently (every 5 questions)
            if question_num % save_every == 0:
                with open(progress_file, 'w', encoding='utf-8') as f:
                    json.dump(categorized, f, indent=2, ensure_ascii=False)
                print(f"   💾 Progress saved ({question_num} questions)\n")
        
        except KeyboardInterrupt:
            print(f"\n⏹️  Interrupted by user at question {question_num}")
            break
        except Exception as e:
            print(f"❌ Error on question {question_num} ('{question['name']}'): {e}")
            # Add with empty categories
            categorized_q = question.copy()
            categorized_q['aiCategories'] = []
            
            if i < len(categorized):
                categorized[i] = categorized_q
            else:
                categorized.append(categorized_q)
    
    # Final save
    with open(progress_file, 'w', encoding='utf-8') as f:
        json.dump(categorized, f, indent=2, ensure_ascii=False)
    
    elapsed = (datetime.now() - start_time).total_seconds()
    processed = len(categorized) - start_from
    
    print(f"\n✅ Batch processing complete!")
    print(f"   Processed: {processed} questions")
    print(f"   Total time: {elapsed/60:.1f} minutes")
    print(f"   Average: {elapsed/processed:.2f}s per question" if processed > 0 else "")
    print(f"   Progress saved to: {progress_file}")
    
    return categorized

print("✅ Safe batch categorization function defined")


✅ Safe batch categorization function defined


In [8]:
# Cell 8: RESTART WITH PROPER RATE LIMITING

# 🚀 SAFE EXECUTION - Conservative rate limiting to avoid API limits
# Saves every 5 questions, can resume from interruptions

if 'questions' in locals():
    print("🎯 Starting SAFE categorization with GPT-4o...")
    print("⚠️  Using conservative 1.5s rate limiting")
    print("💾 Saving every 5 questions for safety")
    print("📊 Strategy-focused analysis with updated categories\n")
    
    # Check if there's existing progress
    progress_file = Path('openai_categorized_progress.json')
    if progress_file.exists():
        with open(progress_file, 'r', encoding='utf-8') as f:
            existing = json.load(f)
        print(f"📂 Found existing progress: {len(existing)} questions completed")
        print("🔄 Will resume from where we left off...\n")
    
    # Run with safe settings
    categorized_questions = categorize_all_questions_safe(
        questions,
        rate_limit_delay=1.5,  # Conservative 1.5 seconds between requests
        save_every=5,          # Save every 5 questions
        start_from=0           # Auto-detects resume point
    )
    
    # Save final results
    with open('openai_categorized_final.json', 'w', encoding='utf-8') as f:
        json.dump(categorized_questions, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Final results saved to openai_categorized_final.json")
    print(f"🎉 Categories updated: 'Algebraic Manipulation' → 'Algebra'")
    print(f"✅ Ready for analysis and database upload!")
else:
    print("⚠️  Run Cell 2 first to load questions")

print("✅ Safe execution cell ready")


🎯 Starting SAFE categorization with GPT-4o...
⚠️  Using conservative 1.5s rate limiting
💾 Saving every 5 questions for safety
📊 Strategy-focused analysis with updated categories

🚀 Starting GPT-4o categorization...
   Questions: 0/1210 (starting from 0)
   Rate limit: 1.5s per request (conservative)
   Save frequency: Every 5 questions
   Estimated time: 30.2 minutes
   Model: GPT-4o with strategy-focused analysis

Processing 1/1210: Place Or Take...
   ✅ Categories: ['Game Theory', 'Combinatorics']
Processing 2/1210: Collecting Toys II...
   ✅ Categories: ['Combinatorics', 'Discrete Random Variables']
Processing 3/1210: Chess Tournament I...
   ✅ Categories: ['Combinatorics']
Processing 4/1210: Poker Hands I...
   ✅ Categories: ['Combinatorics', 'Cards']
Processing 5/1210: Free Sundae...
   ✅ Categories: ['Uniform Random Variables', 'Combinatorics']
   💾 Progress saved (5 questions)

Processing 6/1210: Rubik's Cube Stickers...
   ✅ Categories: ['Geometry', 'Combinatorics']
Processing 

In [ ]:
# Cell 7: RUN FULL CATEGORIZATION WITH GPT-4o

# 🚀 READY TO RUN - Much faster than Gemini!
# OpenAI has higher rate limits, estimated ~12 minutes for 1210 questions

if 'questions' in locals():
    print("🎯 Starting full categorization with GPT-4o...")
    print("📊 Strategy-focused analysis for all 1,210 questions\n")
    
    categorized_questions = categorize_all_questions(
        questions,
        rate_limit_delay=0.1,  # OpenAI allows much higher rates
        save_every=50
    )
    
    # Save final results
    with open('openai_categorized_final.json', 'w', encoding='utf-8') as f:
        json.dump(categorized_questions, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Final results saved to openai_categorized_final.json")
    print(f"🎉 Ready for analysis and database upload!")
else:
    print("⚠️  Run Cell 2 first to load questions")

# Uncomment the line below to run automatically
# print("✅ Execution cell ready - uncomment 'if' condition above to run")


🎯 Starting full categorization with GPT-4o...
📊 Strategy-focused analysis for all 1,210 questions

🚀 Starting GPT-4o categorization of 1210 questions...
   Rate limit: 0.1s per request
   Estimated time: 2.0 minutes
   Model: GPT-4o with strategy-focused analysis

Progress:   25/1210 (  2.1%) | Rate:  80.8 q/min | ETA:  14.7 min
Progress:   50/1210 (  4.1%) | Rate:  81.2 q/min | ETA:  14.3 min
   💾 Progress saved (50 questions)
Progress:   75/1210 (  6.2%) | Rate:  86.1 q/min | ETA:  13.2 min
❌ Error categorizing 'Red And Black Urn II': Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-Lx18Dyukle5ZmWLrgt34p3wW on tokens per min (TPM): Limit 30000, Used 29784, Requested 857. Please try again in 1.282s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
❌ Error categorizing 'Basic Gamma': Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in or

: 

In [ ]:
# Cell 6: Batch Categorization Function

def categorize_all_questions(
    questions: List[Dict],
    rate_limit_delay: float = 0.1,
    save_every: int = 10
) -> List[Dict]:
    """
    Categorize all questions with progress tracking and periodic saving.
    
    Args:
        questions: List of all questions
        rate_limit_delay: Seconds between API calls (0.1s = 600/min, well under OpenAI limits)
        save_every: Save progress every N questions
    
    Returns:
        List of questions with added 'aiCategories' field
    """
    categorized = []
    total = len(questions)
    start_time = time.time()
    
    print(f"🚀 Starting categorization of {total} questions...")
    print(f"   Rate limit: {rate_limit_delay}s per request")
    print(f"   Estimated time: {(total * rate_limit_delay) / 60:.1f} minutes\n")
    
    for i, question in enumerate(questions, 1):
        try:
            categories = categorize_question(question, rate_limit_delay=rate_limit_delay, max_retries=3)
            
            categorized_q = question.copy()
            categorized_q['aiCategories'] = categories
            categorized.append(categorized_q)
            
            # Progress update
            if i % 10 == 0 or i == total:
                elapsed = time.time() - start_time
                rate = i / elapsed
                eta = (total - i) / rate if rate > 0 else 0
                
                print(f"Progress: {i}/{total} ({i/total*100:.1f}%) | "
                      f"Rate: {rate:.1f} q/s | "
                      f"ETA: {eta/60:.1f} min")
            
            # Periodic save
            if i % save_every == 0:
                with open('categorized_questions_progress.json', 'w', encoding='utf-8') as f:
                    json.dump(categorized, f, indent=2, ensure_ascii=False)
                print(f"   💾 Progress saved ({i} questions)")
        
        except Exception as e:
            print(f"❌ Error on question {i} ('{question['name']}'): {e}")
            # Add with empty categories
            categorized_q = question.copy()
            categorized_q['aiCategories'] = []
            categorized.append(categorized_q)
    
    elapsed = time.time() - start_time
    print(f"\n✅ Categorization complete!")
    print(f"   Total time: {elapsed/60:.1f} minutes")
    print(f"   Average: {elapsed/total:.2f}s per question")
    
    return categorized

print("✅ Batch categorization function defined")


✅ Batch categorization function defined


In [ ]:
# Cell 7: RUN FULL CATEGORIZATION

# ⚠️ UNCOMMENT TO RUN - This will take ~20 minutes for 1200 questions
# if 'questions' in locals():
#     categorized_questions = categorize_all_questions(
#         questions,
#         rate_limit_delay=0.1,  # 10 requests/second, well under OpenAI limits
#         save_every=10
#     )
#     
#     # Save final results
#     with open('categorized_questions_final.json', 'w', encoding='utf-8') as f:
#         json.dump(categorized_questions, f, indent=2, ensure_ascii=False)
#     
#     print("\n💾 Final results saved to categorized_questions_final.json")
# else:
#     print("⚠️  Run Cell 2 first")

print("⚠️  Cell ready - uncomment to run full categorization")


⚠️  Cell ready - uncomment to run full categorization


In [ ]:
# Cell 8: Batch Categorization Function

def categorize_all_questions(
    questions: List[Dict], 
    fixed_categories: List[str],
    rate_limit_delay: float = 0.1,
    save_every: int = 50
) -> List[Dict]:
    """
    Categorize all questions with progress tracking and periodic saving.
    
    Args:
        questions: List of all questions
        fixed_categories: Fixed list of categories from Phase 1
        rate_limit_delay: Seconds between API calls (0.1s = 600/min, well under OpenAI limits)
        save_every: Save progress every N questions
    
    Returns:
        List of questions with added 'aiCategories' field
    """
    from datetime import datetime
    
    categorized = []
    total = len(questions)
    start_time = time.time()
    
    print(f"🚀 Starting categorization of {total} questions...")
    print(f"   Rate limit: {rate_limit_delay}s per request")
    print(f"   Estimated time: {(total * rate_limit_delay) / 60:.1f} minutes\n")
    
    for i, question in enumerate(questions, 1):
        try:
            categories = categorize_question(question, fixed_categories, rate_limit_delay)
            
            categorized_q = question.copy()
            categorized_q['aiCategories'] = categories
            categorized.append(categorized_q)
            
            # Progress update
            if i % 10 == 0 or i == total:
                elapsed = time.time() - start_time
                rate = i / elapsed
                eta = (total - i) / rate if rate > 0 else 0
                
                print(f"Progress: {i}/{total} ({i/total*100:.1f}%) | "
                      f"Rate: {rate:.1f} q/s | "
                      f"ETA: {eta/60:.1f} min")
            
            # Periodic save
            if i % save_every == 0:
                with open('categorized_questions_progress.json', 'w', encoding='utf-8') as f:
                    json.dump(categorized, f, indent=2, ensure_ascii=False)
                print(f"   💾 Progress saved ({i} questions)")
        
        except Exception as e:
            print(f"❌ Error on question {i} ('{question['name']}'): {e}")
            # Add with fallback categories
            categorized_q = question.copy()
            categorized_q['aiCategories'] = question['tags']
            categorized.append(categorized_q)
    
    elapsed = time.time() - start_time
    print(f"\n✅ Categorization complete!")
    print(f"   Total time: {elapsed/60:.1f} minutes")
    print(f"   Average: {elapsed/total:.2f}s per question")
    
    return categorized

print("✅ Batch categorization function defined")


✅ Batch categorization function defined


In [ ]:
# Cell 9: RUN FULL CATEGORIZATION

# ⚠️ UNCOMMENT TO RUN - This will take ~20 minutes for 1200 questions
# if 'FIXED_CATEGORIES' in locals() and 'questions' in locals():
#     categorized_questions = categorize_all_questions(
#         questions, 
#         FIXED_CATEGORIES,
#         rate_limit_delay=0.1,  # 10 requests/second, well under OpenAI limits
#         save_every=50
#     )
#     
#     # Save final results
#     with open('categorized_questions_final.json', 'w', encoding='utf-8') as f:
#         json.dump(categorized_questions, f, indent=2, ensure_ascii=False)
#     
#     print("\n💾 Final results saved to categorized_questions_final.json")
# else:
#     print("⚠️  Run Cells 2 and 5 first")

print("⚠️  Cell ready - uncomment to run full categorization")


In [ ]:
# Cell 10: Analyze Category Distribution

def analyze_categories(categorized_questions: List[Dict]) -> Dict:
    """
    Analyze the distribution of categories across all questions.
    """
    from collections import Counter
    
    all_categories = []
    categories_per_question = []
    
    for q in categorized_questions:
        cats = q.get('aiCategories', [])
        all_categories.extend(cats)
        categories_per_question.append(len(cats))
    
    category_counts = Counter(all_categories)
    
    print(f"📊 Category Analysis")
    print(f"{'='*80}\n")
    
    print(f"Total questions: {len(categorized_questions)}")
    print(f"Total unique categories: {len(category_counts)}")
    print(f"Average categories per question: {sum(categories_per_question)/len(categories_per_question):.1f}")
    print(f"Min categories: {min(categories_per_question)}")
    print(f"Max categories: {max(categories_per_question)}\n")
    
    print(f"Top 20 Most Common Categories:")
    print(f"{'-'*80}")
    for i, (cat, count) in enumerate(category_counts.most_common(20), 1):
        pct = count / len(categorized_questions) * 100
        print(f"{i:2d}. {cat:30s} {count:4d} questions ({pct:5.1f}%)")
    
    return {
        'total_questions': len(categorized_questions),
        'unique_categories': len(category_counts),
        'avg_categories_per_question': sum(categories_per_question)/len(categories_per_question),
        'category_counts': dict(category_counts)
    }

print("✅ Analysis function defined")


In [ ]:
# Cell 11: Export for Database Upload

def export_for_database(categorized_questions: List[Dict], output_file: str = 'categorized_for_db.json'):
    """
    Export categorized questions in format ready for Supabase upload.
    """
    db_ready = []
    
    for q in categorized_questions:
        db_ready.append({
            'url_ending': q['link'].replace('/questions/', ''),
            'name': q['name'],
            'aiCategories': q.get('aiCategories', [])
        })
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(db_ready, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Exported {len(db_ready)} questions to {output_file}")
    print(f"   Ready for database upload!")
    
    return db_ready

print("✅ Export function defined")
